In [1]:
#Imports
import foursquare
import pandas as pd
import numpy as np
from scripts.constants import *
from scripts import dataframe_builder, api_request

In [ ]:
# Collect new random samples
api_request.collect_random_samples(100)

In [2]:
# Reconcile dataset with logged API responses
dataframe_builder.update_dataframe()

In [28]:
# Make requests for missing information
api_request.request_missing_information()

In [92]:
pd.read_csv(SAVE_DIR+RESULTS, index_col=0)

,country,city,yelp,four_square,google
"-44.48767, -65.31431",Argentina,Puerto Larrea,0.0,0.0,0.0
"17.55, 32.95",Sudan,Jabal Sa’ada,0.0,0.0,0.0
"-22.2, 131.06667",Australia,Matchee Spring,0.0,0.0,2.0
"52.03982, 0.72503",United Kingdom,Sudbury,24.0,30.0,20.0
"48.953, 3.06238",France,La Ferte-sous-Jouarre,2.0,30.0,20.0
...,...,...,...,...,...
"4.37577, -54.30529",France,Grand Bala,0.0,1.0,0.0
"35.28852, 109.8059",China,Xichengcun,0.0,0.0,0.0
"-1.83217, 130.12765",Indonesia,Tip,0.0,1.0,0.0
"65.75287, 49.8525",Russia,Kosma,0.0,0.0,0.0


In [130]:
df = pd.read_csv(SAVE_DIR+RESULTS, index_col=0)
codes = pd.read_csv(SAVE_DIR+"country_codes.csv")
codes.rename(columns={'name':'country','alpha-3':'code'}, inplace=True)
# df = df.merge(codes[['country','code']], on='country')

# df['total'] = df['yelp'] + df['four_square'] + df['google']
df = pd.merge(df, codes[['country','code']], on='country', how='left')

In [131]:
df[df['code'].isna()]['country'].unique()

array(['United Kingdom', nan, 'Iran', 'Russia', 'United States',
       'Visayas', 'Democratic Republic of the Congo', 'North Korea',
       'Cape Verde', 'Venezuela', 'Bolivia', 'Syria', 'Vietnam', 'Laos',
       'Tanzania'], dtype=object)

In [132]:
df.loc[df['country'] == 'United Kingdom', 'code'] = 'GBR'
df.loc[df['country'] == 'Iran', 'code'] = 'IRN'
df.loc[df['country'] == 'Russia', 'code'] = 'RUS'
df.loc[df['country'] == 'United States', 'code'] = 'USA'
df.loc[df['country'] == 'Visayas', 'code'] = 'PHL'
df.loc[df['country'] == 'Democratic Republic of the Congo', 'code'] = 'COD'
df.loc[df['country'] == 'North Korea', 'code'] = 'PRK'
df.loc[df['country'] == 'Cape Verde', 'code'] = 'CPV'
df.loc[df['country'] == 'Venezuela', 'code'] = 'VEN'
df.loc[df['country'] == 'Bolivia', 'code'] = 'BOL'
df.loc[df['country'] == 'Syria', 'code'] = 'SYR'
df.loc[df['country'] == 'Vietnam', 'code'] = 'VNM'
df.loc[df['country'] == 'Laos', 'code'] = 'LAO'
df.loc[df['country'] == 'Tanzania', 'code'] = 'TZA'


In [137]:
df.dropna()

,country,city,yelp,four_square,google,code
0,Argentina,Puerto Larrea,0.0,0.0,0.0,ARG
1,Sudan,Jabal Sa’ada,0.0,0.0,0.0,SDN
2,Australia,Matchee Spring,0.0,0.0,2.0,AUS
3,United Kingdom,Sudbury,24.0,30.0,20.0,GBR
4,France,La Ferte-sous-Jouarre,2.0,30.0,20.0,FRA
...,...,...,...,...,...,...
388,France,Grand Bala,0.0,1.0,0.0,FRA
389,China,Xichengcun,0.0,0.0,0.0,CHN
390,Indonesia,Tip,0.0,1.0,0.0,IDN
391,Russia,Kosma,0.0,0.0,0.0,RUS


In [138]:
### Compare Coverage
fs = df[['country','four_square']].groupby('country').sum()
fs_countries = len(fs[fs['four_square']!= 0])
fs_venues = sum(fs['four_square'])

yelp = df[['country','yelp']].groupby('country').sum()
yelp_countries = len(yelp[yelp['yelp']!= 0])
yelp_venues = sum(yelp['yelp'])

google = df[['country','google']].groupby('country').sum()
google_countries = len(google[google['google']!= 0])
google_venues = sum(google['google'])

samples = len(df)
print(f''' 
      From {samples} samples:''')

print(f''' 
      Four Square:
      {fs_venues} venues across {fs_countries} countries
      ''')
print(f''' 
      Google:
      {google_venues} venues across {google_countries} countries
      ''')
print(f''' 
      Yelp:
      {yelp_venues} venues across {yelp_countries} countries
      ''')

 
      From 393 samples:
 
      Four Square:
      2386.0 venues across 48 countries
      
 
      Google:
      158.0 venues across 15 countries
      
 
      Yelp:
      276.0 venues across 11 countries
      


In [154]:
df['total'] = df['yelp'] + df['four_square'] + df['google']
sum_df = df.groupby('code').sum().sort_values('total', ascending=False )
avg_df = df.groupby('code').mean().sort_values('total', ascending=False )

In [155]:
sum_df[sum_df['yelp'] != 0]

,yelp,four_square,google,total
code,,,,
USA,94.0,394.0,0.0,488.0
DEU,99.0,90.0,20.0,209.0
CAN,8.0,131.0,20.0,159.0
JPN,35.0,120.0,0.0,155.0
GBR,25.0,60.0,20.0,105.0
NZL,5.0,72.0,0.0,77.0
AUS,1.0,64.0,2.0,67.0
DNK,3.0,60.0,0.0,63.0
FRA,2.0,31.0,20.0,53.0


In [156]:
avg_df[avg_df['yelp']!= 0]

,yelp,four_square,google,total
code,,,,
DEU,33.000000,30.000000,6.666667,69.666667
GBR,12.500000,30.000000,10.000000,52.500000
JPN,8.750000,30.000000,0.000000,38.750000
MYS,3.000000,30.000000,0.000000,33.000000
DNK,1.500000,30.000000,0.000000,31.500000
ESP,1.000000,30.000000,0.000000,31.000000
NZL,1.666667,24.000000,0.000000,25.666667
FRA,0.666667,10.333333,6.666667,17.666667
USA,3.032258,12.709677,0.000000,15.741935


In [321]:
import plotly.graph_objects as go
import pandas as pd

# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
def display_company_choropleth(company, color):
    fig = go.Figure(data=go.Choropleth(
        locations = avg_df[avg_df['yelp']!= 0].index,
        z = avg_df[avg_df['yelp']!= 0][company],
        text = df['code'],
        colorscale = color,
        # autocolorscale=False,
        # reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        # colorbar_tickprefix = '$',
        colorbar_title = 'Average results per request',
    ))

    fig.update_layout(
        width=1000,
        autosize=True,
        margin=dict(t=50, b=50, l=0, r=0),
        title_text=f'{company} performance',
        geo=dict(
            bgcolor = '#D4D4D4',
            landcolor = '#D4D4D4',
            lakecolor = '#D4D4D4',
            visible=True,
            showframe=True,
            showcoastlines=True,
            projection_type='equirectangular',
            # framewidth=600
                 ),

    
                )

    fig.data[0].colorbar.x=.9
    fig.data[0].colorbar.y=.59
    fig.show()

In [324]:
display_company_choropleth('four_square','Reds')

In [325]:
display_company_choropleth('google','Blues')

In [326]:
display_company_choropleth('yelp','Greens')

In [ ]:
    add_traces(go.Choropleth(locations=Country_df['Country_code'][Country_df["Country"]==country],
                            z = [1],
                            colorscale = [[0, col_swe],[1, col_swe]],
                            colorbar=None,
                            showscale = False))
    
    